### Оглавление:
1. [Подготовка данных](#start)
2. [Обучение моделей](#models)
3. [Анализ моделей](#analisys)

# Подготовка данных
<a id="start"></a>

In [1]:
!pip install lightgbm

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor


try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('autos.csv')
data_initial = data
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [3]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


В данных присутсвует достаточно большое количество пропусков, а также аномальные сначения (например, год регистрации 9999 или мощность 0). Кроме этого у числовых признаков наблюдаются разные по масштабу разбросы значений.

Для начала избавимся от признаков, которые не влияют на стоимость автомобиля: DateCrawled, RegistrationMonth, DateCreated, NumberOfPictures, PostalCode, LastSeen.

In [4]:
data = data.drop(columns=['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
Price               354369 non-null int64
VehicleType         316879 non-null object
RegistrationYear    354369 non-null int64
Gearbox             334536 non-null object
Power               354369 non-null int64
Model               334664 non-null object
Kilometer           354369 non-null int64
FuelType            321474 non-null object
Brand               354369 non-null object
NotRepaired         283215 non-null object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


Пропуски в Model заполним модами этого значения среди марок автомобилей.

In [5]:
brands_models = data.groupby('Brand')['Model'].apply(lambda x: x.mode())
data['Model'] = data['Model'].fillna(data['Brand'].map(brands_models[:,0]))

Пропуски в VehicleType заполним модами этого значения среди моделей автомобилей.

In [6]:
models_vehicle_types = data.groupby('Model')['VehicleType'].apply(lambda x: x.mode())
data['VehicleType'] = data['VehicleType'].fillna(data['Model'].map(models_vehicle_types[:,0]))

Также модами среди моделей автомобилей заполним значения Gearbox и FuelType.

In [7]:
models_gearboxes = data.groupby('Model')['Gearbox'].apply(lambda x: x.mode())
data['Gearbox'] = data['Gearbox'].fillna(data['Model'].map(models_gearboxes[:,0]))

models_fuel_types = data.groupby('Model')['FuelType'].apply(lambda x: x.mode())
data['FuelType'] = data['FuelType'].fillna(data['Model'].map(models_fuel_types[:,0]))

Пропуски в признаке NotRepaired заполним значениями no, предполагая, что пропуски в данном поле пользователи оставляли, так как машина в ремонте не была.

In [8]:
data['NotRepaired'] = data['NotRepaired'].fillna('no')

Строки, в которых не удалось заполнить пропуски в признаках, удалим.

In [9]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350995 entries, 0 to 354368
Data columns (total 10 columns):
Price               350995 non-null int64
VehicleType         350995 non-null object
RegistrationYear    350995 non-null int64
Gearbox             350995 non-null object
Power               350995 non-null int64
Model               350995 non-null object
Kilometer           350995 non-null int64
FuelType            350995 non-null object
Brand               350995 non-null object
NotRepaired         350995 non-null object
dtypes: int64(4), object(6)
memory usage: 29.5+ MB


Также избавимся от явно аномальных значений: записей со значениями Power меньше 20 и больше 600, со значениями RegistrationYear меньше 1950 и больше 2021.

In [10]:
data = data[(data['Power'] >= 20) & (data['Power'] <= 600)]

data = data[(data['RegistrationYear'] >= 1950) & (data['RegistrationYear'] <= 2021)]

data.describe()

,Price,RegistrationYear,Power,Kilometer
count,311501.000000,311501.000000,311501.000000,311501.000000
mean,4690.604322,2003.314936,120.047512,128736.231986
std,4579.874313,6.812481,53.410670,36554.595321
min,0.000000,1950.000000,20.000000,5000.000000
25%,1250.000000,1999.000000,75.000000,125000.000000
50%,2999.000000,2003.000000,110.000000,150000.000000
75%,6800.000000,2008.000000,150.000000,150000.000000
max,20000.000000,2019.000000,600.000000,150000.000000


Также в данных присутствует некоторое количество объектов с значением Price равным 0, либо с символической ценой (1, 2). Предположим, что это случаи, в которых клиенты расчитывают избавиться от ненужного им автомобиля, чтобы не платить за него налоги и утилизационный сбор. 

In [11]:
print(f'Потеря данных после прдобработки составила {len(data_initial)-len(data)} строк или же {round(100-(100*len(data)/len(data_initial)), 2)}%')

Потеря данных после прдобработки составила 42868 строк или же 12.1%


# Обучение моделей
<a id="models"></a>

### Подготовка признаков.

In [12]:
data.select_dtypes(include='object').nunique()

VehicleType      8
Gearbox          2
Model          249
FuelType         7
Brand           39
NotRepaired      2
dtype: int64

Отделим целевой признак от остальных, а затем проведем прямое кодирование категориальных признаков кроме Model и Brand (так как они имеют слишком большое количество уникальных значений, закодируем их порядковым кодированием).

In [13]:
data = data.reset_index()
X, y = data.drop(columns='Price'), data['Price']

encoder = OrdinalEncoder()
Xoe = pd.DataFrame(encoder.fit_transform(X[['Model', 'Brand']]), columns=['Model', 'Brand'])

Xohe = pd.get_dummies(X[['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Kilometer', 'FuelType', 'NotRepaired']], drop_first=True)

X = Xoe.merge(Xohe, left_index=True, right_index=True)
X

,Model,Brand,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,Gearbox_manual,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol,NotRepaired_yes
0,29.0,1.0,2011,190,125000,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1
1,117.0,14.0,2004,163,125000,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,116.0,37.0,2001,75,150000,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
3,101.0,31.0,2008,69,90000,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,11.0,2.0,1995,102,150000,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311496,4.0,0.0,2016,150,150000,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
311497,140.0,30.0,2004,225,150000,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1
311498,106.0,32.0,2000,101,125000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
311499,223.0,37.0,1996,102,150000,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


Разделим данные на обучающую и тестовую выборки в соотношении 3:1.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

def check_size(name, sample):
    print(f'Размер {name} выборки датасета: {len(sample)}')
check_size('обучающей', y_train)
check_size('тестовой', y_test)

Размер обучающей выборки датасета: 233625
Размер тестовой выборки датасета: 77876


В данных присутствуют количественные признаки с разными разбросами значений, так что стандартизируем эти данные в обучающей и тестовой выборках, предварительно настроив стандартизатор на обучающей.

In [15]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']
scaler = StandardScaler()
scaler.fit(X_train.loc[:,numeric])

pd.options.mode.chained_assignment = None 

X_train.loc[:,numeric] = scaler.transform(X_train.loc[:,numeric])
X_test.loc[:,numeric] = scaler.transform(X_test.loc[:,numeric])

### Обучение линейной регрессии.

Построим модель линейной регрессии, проверим её на кросс-валидации и высяним скорость ее обучения.

In [16]:
%%time

model_lin_reg = LinearRegression()
model_lin_reg.fit(X_train, y_train)

MSE_lin_reg = np.mean(cross_validate(model_lin_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')['test_score'])

print(f'Значение RMSE линейной регрессии при проверке на кросс-валидации: {np.abs(MSE_lin_reg)**0.5}')

Значение RMSE линейной регрессии при проверке на кросс-валидации: 2975.1866104003134
Wall time: 568 ms


Проверим скорость предсказания линейной регрессии.

In [17]:
%%time

model_lin_reg.predict(X_test)

Wall time: 8.01 ms


array([4460.46553216, 4713.10917276,  360.95054938, ..., 9749.08436311,
       4496.63115686, 1192.72336512])

### Обучение случайного леса.

Построим модель случайного леса с подбором параметров через RandomizedSearch и выясним скорость его обучения.

In [18]:
%%time

RS_params = {'max_depth': range(1, 10),
             'n_estimators': range(1, 51, 10)}

RS_random_forest = RandomizedSearchCV(RandomForestRegressor(random_state=1),
                                      param_distributions=RS_params,
                                      scoring='neg_mean_squared_error', cv=5,
                                      n_iter=20, random_state=1)
RS_random_forest.fit(X_train, y_train)

MSE_random_forest = RS_random_forest.best_score_

print(f'Лучшее RMSE случайного леса: {np.abs(MSE_random_forest)**0.5}')
print(f'Параметры модели случайного леса: {RS_random_forest.best_params_}')

Лучшее RMSE случайного леса: 2036.0290248647348
Параметры модели случайного леса: {'n_estimators': 41, 'max_depth': 9}
Wall time: 5min 41s


Проверим скорость предсказания случайного леса.

In [19]:
%%time

RS_random_forest.predict(X_test)

Wall time: 191 ms


array([ 2575.57017403,  2310.3395517 ,  1002.58809939, ...,
       10972.74917302,  1653.95833719,  1544.47089106])

### Проверка на адекватность.

Сравним метрики полученных моделей с метрикой модели, предсказывающей только среднее значение целевого признака.

In [20]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)
prediction_dummy_mean = dummy_regr.predict(X_test)

print(f'RMSE для константной модели со средними значениями: {mean_squared_error(y_test, prediction_dummy_mean)**0.5}')

RMSE для константной модели со средними значениями: 4573.358848031977


Значение RMSE у dummy-модели получилось сильно больше, так что можно говорить об адекватности наших линейной регрессии и случайного леса.

### Обучение градиентного бустинга.

Построим модель градиентного бустинга из библиотеки LightGBM с изменением нескольких параметров, определим скорость обучения и скорость предсказания.

In [21]:
%%time

train_features, valid_features, train_target, valid_target = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
train_data_lgb = lgb.Dataset(train_features, train_target)
valid_data_lgb = lgb.Dataset(valid_features, valid_target, reference=train_data_lgb)

parameters1 = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'learning_rate': 0.05,
    'verbose': -1
}

model_lgb1 = lgb.train(parameters1, train_data_lgb, 5000, valid_sets=valid_data_lgb, verbose_eval=False)

Wall time: 11.8 s


In [22]:
%%time

model_lgb_prediction1 = model_lgb1.predict(valid_features)

print(f'Значение RMSE градиентного бустинга из библиотеки LightGBM с первым набором параметров: {mean_squared_error(valid_target, model_lgb_prediction1)**0.5}')

Значение RMSE градиентного бустинга из библиотеки LightGBM с первым набором параметров: 1658.0040978784587
Wall time: 1.54 s


Изменим параметры бустинга и проверим, как это отразится на результате.

In [23]:
%%time

parameters2 = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'num_leaves': 61,
    'feature_fraction': 1,
    'learning_rate': 0.1,
    'verbose': -1
}

model_lgb2 = lgb.train(parameters2, train_data_lgb, 5000, valid_sets=valid_data_lgb, verbose_eval=False)

Wall time: 12.6 s


In [24]:
%%time

model_lgb_prediction2 = model_lgb2.predict(valid_features)

print(f'Значение RMSE градиентного бустинга из библиотеки LightGBM со вторым набором параметров: {mean_squared_error(valid_target, model_lgb_prediction2)**0.5}')

Значение RMSE градиентного бустинга из библиотеки LightGBM со вторым набором параметров: 1671.831217534069
Wall time: 2.65 s


# Анализ моделей
<a id="analisys"></a>

Наилучший результат показала модель градиентного бустинга с первым набором параметров. Проверим ее RMSE на тестовой выборке.

In [25]:
model_lgb_prediction1_test = model_lgb1.predict(X_test)

print(f'Значение RMSE градиентного бустинга из библиотеки LightGBM с первым набором параметров: {mean_squared_error(y_train, model_lgb_prediction1_test)**0.5}')

Значение RMSE градиентного бустинга из библиотеки LightGBM с первым набором параметров: 1483.337710167307


### Выводы.

Из моделей случайного леса и линейной регрессии наилучший результат показал случайный лес (2975 у регрессии против 2036 у случайного леса), но при этом он затрачивает на обучение ~6 минут (при подборе параметров через RandomSearch), тогда как линейная регрессия отрабатывает почти мгновенно. Однако обе эти простые модели проигрывают модели градиентного бустинга на основе решающего дерева, который показывает результат RMSE на валидационной выборке в 1658, и еще более хороший результат на тестовой выборке, при времени обучения в несколько десятков секунд в зависимости от параметров и числа итераций. Что касается скорости предсказаний, то здесь быстрее (меньше секунды) отрабатывают линейная регрессия и случайный лес, а градиентный бустинг предсказывает несколько секунд в зависимости от параметров.